## Import Dependancies

In [1]:
import pandas as pd
import json
import gzip

## Extract and Read CSVs

In [2]:
def csv_to_df(csv_path):
    df = pd.read_csv(csv_path, compression='gzip', dtype=object)
    return df

## Decoding JSONs 

In [3]:
#Decoding JSON
def code_map(json_path):
    with open(json_path, "r") as f_open:
        code_map = json.load(f_open)
    return code_map

## Cleaning up Data
1. Find out Column info
2. Drop irrelavent columns 
3. Replace the Blanks in the Columns 
4. Replace values with the codes on JSON
5. Rename Columns to improve readability

In [4]:
def reduce_df(df):
    #Drop Columns that are irrelavent to hypothesis
    reduced_df = df.drop(columns=['113_cause_recode', '39_cause_recode','resident_status',
                                          'education_1989_revision', 'age_recode_52', 'age_recode_27', 'detail_age',
                                          'education_reporting_flag', 'age_substitution_flag', 'infant_age_recode_22',
                                          'place_of_death_and_decedents_status', 'injury_at_work', 'manner_of_death',
                                          'method_of_disposition', 'autopsy', 'activity_code', 'race',
                                          'place_of_injury_for_causes_w00_y34_except_y06_and_y07_',
                                          'bridged_race_flag', 'race_imputation_flag', '130_infant_cause_recode',
                                          'race_recode_3', 'hispanic_origin',
                                          'detail_age_type', 'number_of_entity_axis_conditions', 'entity_condition_1',
                                          'entity_condition_2', 'entity_condition_3', 'entity_condition_4',
                                          'entity_condition_5', 'entity_condition_6', 'entity_condition_7',
                                          'entity_condition_8', 'entity_condition_9', 'entity_condition_10',
                                          'entity_condition_11', 'entity_condition_12', 'entity_condition_13',
                                          'entity_condition_14', 'entity_condition_15', 'entity_condition_16',
                                          'entity_condition_17', 'entity_condition_18', 'entity_condition_19',
                                          'entity_condition_20', 'number_of_record_axis_conditions', 'record_condition_1',
                                          'record_condition_2', 'record_condition_3', 'record_condition_4',
                                          'record_condition_5', 'record_condition_6', 'record_condition_7',
                                          'record_condition_8', 'record_condition_9', 'record_condition_10',
                                          'record_condition_11', 'record_condition_12', 'record_condition_13',
                                          'record_condition_14', 'record_condition_15', 'record_condition_16',
                                          'record_condition_17', 'record_condition_18', 'record_condition_19',
                                          'record_condition_20'])
    #Drop the NAN and unknown values
    reduced_df = reduced_df.dropna(subset=['education_2003_revision'])
    reduced_df = reduced_df.loc[reduced_df['education_2003_revision'] != 'Unknown']
    return reduced_df

In [5]:
#Using JSON decoder, replace values in the dataframes. 
#Go through ALL columns
def recode_values(df, code_map):
    recoded_df = df.copy()
    recoded_df['education_2003_revision'] = recoded_df['education_2003_revision'].apply(
        lambda x: code_map['education_2003_revision'][x])
    recoded_df['month_of_death'] = recoded_df['month_of_death'].apply(
        lambda x: code_map['month_of_death'][x])
    recoded_df['age_recode_12'] = recoded_df['age_recode_12'].apply(
    lambda x: code_map['age_recode_12'][x])
    recoded_df['marital_status'] = recoded_df['marital_status'].apply(
        lambda x: code_map['marital_status'][x])
    recoded_df['race_recode_5'] = recoded_df['race_recode_5'].apply(
        lambda x: code_map['race_recode_5'][x])
    recoded_df['hispanic_originrace_recode'] = recoded_df['hispanic_originrace_recode'].apply(
        lambda x: code_map['hispanic_originrace_recode'][x])
    recoded_df['day_of_week_of_death'] = recoded_df['day_of_week_of_death'].apply(
        lambda x: code_map['day_of_week_of_death'][x])
    recoded_df['358_cause_recode'] = recoded_df['358_cause_recode'].apply(
        lambda x: code_map['358_cause_recode'][x].split('(')[0].replace('=',"'"))
    return recoded_df

In [6]:
def icd_chapters(icd_code):
    icd_letter = icd_code[0]
    icd_num = int(icd_code[1:3])
    if icd_letter == 'A' or icd_letter == 'B':
        return 'Certain infectious and parasitic diseases'
    elif icd_letter == 'C':
        return 'Neoplasms'
    elif icd_letter == 'D':
        if icd_num <= 49:
            return 'Neoplasms'
        else:
            return 'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism'
    elif icd_letter == 'E':
        return 'Endocrine, nutritional and metabolic diseases'
    elif icd_letter == 'F':
        return 'Mental, behavioral and neurodevelopmental disorders'
    elif icd_letter == 'G':
        return 'Diseases of the nervous system'
    elif icd_letter == 'H':
        if icd_num <= 59:
            return 'Diseases of the eye and adnexa'
        else:
            return 'Diseases of the ear and mastoid process'
    elif icd_letter == 'I':
        return 'Diseases of the circulatory system'
    elif icd_letter == 'J':
        return 'Diseases of the respiratory system'
    elif icd_letter == 'K':
        return 'Diseases of the digestive system'
    elif icd_letter == 'L':
        return 'Diseases of the skin and subcutaneous tissue'
    elif icd_letter == 'M':
        return 'Diseases of the musculoskeletal system and connective tissue'
    elif icd_letter == 'N':
        return 'Diseases of the genitourinary system'
    elif icd_letter == 'O':
        return 'Pregnancy, childbirth and the puerperium'
    elif icd_letter == 'P':
        return 'Certain conditions originating in the perinatal period'
    elif icd_letter == 'Q':
        return 'Congenital malformations, deformations and chromosomal abnormalities'
    elif icd_letter == 'R':
        return 'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified'
    elif icd_letter == 'S' or icd_letter == 'T':
        return 'Injury, poisoning and certain other consequences of external causes'
    elif icd_letter == 'U':
        return 'Codes for special purposes'
    elif icd_letter == 'V' or icd_letter == 'W' or icd_letter == 'X' or icd_letter == 'Y':
        return 'External causes of morbidity and mortality'
    elif icd_letter == 'Z':
        return 'Factors influencing health status and contact with health services'
    else:
        return float("NaN")

In [7]:
#icd_code_10th_revision column changes to icd_code_10 in 2012
def replace_icd_chapters(df):
    icd_code_chapters = []
    icd_replaced_df = df.copy()
    for index, row in icd_replaced_df.iterrows():
        icd_code = row['icd_code_10th_revision']
        icd_code_chapters.append(icd_chapters(icd_code))
    icd_replaced_df['Cause of Death Category'] = icd_code_chapters
    return icd_replaced_df

In [8]:
#Rename Columns for additional readability
def rename_columns(df):
    renamed_df = df.copy()
    renamed_df = renamed_df.rename(columns={"education_2003_revision": "Education Level", 
                                                      "month_of_death": "Month of Death", 
                                                      "sex": "Sex/Gender",  
                                                      "age_recode_12": "Age Groups", 
                                                      "infant_age_recode_22": "Infant Age Groups", 
                                                      "marital_status": "Marital Status", 
                                                      "current_data_year": "Year", 
                                                      "icd_code_10th_revision": "ICD Code",
                                                      "icd_code_10": "ICD Code",
                                                      "358_cause_recode": "Cause of Death",
                                                      "race_recode_5": "Race",
                                                      "hispanic_originrace_recode": "Hispanic Origin",
                                                      "day_of_week_of_death": "Day of Week"})
    return renamed_df

In [9]:
cod_df = pd.DataFrame()
years = [str(2000+x) for x in range(5,16)]
for year in years:
    csv_path = f"Raw_CSVs/kaggle_data/{year}_data.csv.gz"
    json_path = f"Raw_CSVs/kaggle_data/{year}_codes.json"
    
    df = csv_to_df(csv_path)
    column_map = code_map(json_path)
    
    if year == '2012':
        df = df.rename(columns={"icd_code_10": "icd_code_10th_revision"})
    
    df = reduce_df(df)
    df = recode_values(df, column_map)
    df = replace_icd_chapters(df)
    df = rename_columns(df)
    
    cod_df = pd.concat([cod_df, df], ignore_index=True)
cod_df

,Education Level,Month of Death,Sex/Gender,Age Groups,Marital Status,Day of Week,Year,ICD Code,Cause of Death,Race,Hispanic Origin,Cause of Death Category
0,8th grade or less,June,M,85 years and over,Married,Saturday,2005,I251,All other forms of chronic ischemic heart dise...,White,Mexican,Diseases of the circulatory system
1,"9 - 12th grade, no diploma",January,F,45 - 54 years,Married,Saturday,2005,J449,Other chronic obstructive pulmonary disease,White,Non - Hispanic white,Diseases of the respiratory system
2,high school graduate or GED completed,January,F,65 - 74 years,Widowed,Sunday,2005,C349,"Of trachea, bronchus and lung",White,Non - Hispanic white,Neoplasms
3,high school graduate or GED completed,January,M,55 - 64 years,Married,Monday,2005,X72,Intentional self-harm,White,Non - Hispanic white,External causes of morbidity and mortality
4,high school graduate or GED completed,January,M,75 - 84 years,Married,Sunday,2005,I64,"Stroke, not specified as hemorrhage or infarct...",White,Non - Hispanic white,Diseases of the circulatory system
...,...,...,...,...,...,...,...,...,...,...,...,...
19525729,8th grade or less,May,M,Under 1 year (includes not stated infant ages),"Never married, single",Saturday,2015,P072,Disorders related to short gestation and low b...,Black,Hispanic origin unknown,Certain conditions originating in the perinata...
19525730,Unknown,December,F,45 - 54 years,Marital Status unknown,Friday,2015,I251,All other forms of chronic ischemic heart dise...,White,Non - Hispanic white,Diseases of the circulatory system
19525731,Unknown,December,M,25 - 34 years,Marital Status unknown,Sunday,2015,F102,Mental and behavioral disorders due to use of ...,White,Other or unknown Hispanic,"Mental, behavioral and neurodevelopmental diso..."
19525732,Unknown,July,M,75 - 84 years,Marital Status unknown,Saturday,2015,I250,"Atherosclerotic cardiovascular disease, so des...",Black,Non - Hispanic black,Diseases of the circulatory system


In [10]:
condition_1 = (cod_df['Cause of Death'] != 'Other ill-defined and unknown causes of mortality ')
condition_2 = (cod_df['Cause of Death'] != 'Ill-defined and unknown causes of mortality ')
#condition_3 = (cod_df['Education Level'] != 'Unknown')
#condition_4 = (cod_df['Day of Week'] != 'Unknown')
#condition_5 = (cod_df['Hispanic Origin'] != 'Hispanic origin unknown')
cod_df = cod_df.loc[(condition_1 & condition_2)]
cod_df.head()

,Education Level,Month of Death,Sex/Gender,Age Groups,Marital Status,Day of Week,Year,ICD Code,Cause of Death,Race,Hispanic Origin,Cause of Death Category
0,8th grade or less,June,M,85 years and over,Married,Saturday,2005,I251,All other forms of chronic ischemic heart dise...,White,Mexican,Diseases of the circulatory system
1,"9 - 12th grade, no diploma",January,F,45 - 54 years,Married,Saturday,2005,J449,Other chronic obstructive pulmonary disease,White,Non - Hispanic white,Diseases of the respiratory system
2,high school graduate or GED completed,January,F,65 - 74 years,Widowed,Sunday,2005,C349,"Of trachea, bronchus and lung",White,Non - Hispanic white,Neoplasms
3,high school graduate or GED completed,January,M,55 - 64 years,Married,Monday,2005,X72,Intentional self-harm,White,Non - Hispanic white,External causes of morbidity and mortality
4,high school graduate or GED completed,January,M,75 - 84 years,Married,Sunday,2005,I64,"Stroke, not specified as hemorrhage or infarct...",White,Non - Hispanic white,Diseases of the circulatory system


In [12]:
cod_df.to_csv('data/cod_clean.csv.gz', index=False, header=True, compression='gzip')